In [ ]:
operators={"UAdd":"+",
            "USub":"-",
            "Sub":"-",
            "Add":'+',
            "Mult":"*",
            "Div":"/",
            "FloorDiv":"//",
            "Pow":'**',
            'Mod':'%',
            'Gt':'>',
            'Lt':'<',
            'GtE':'>=',
            'LtE':'<=',
            'Eq':'==',
            'NotEq':'!=',
            'And':'and',
            'Or':'or',
            'NotIn':'not in',
            'Not':'not',
            'Is':'is',
            'IsNot':'is not',
            'BitAnd':'&',
            'BitOr':'|',
            'Invert':'~',
            'BitXor':'^',
            'RShift':'>>',
            'LShift':'<<'
            };

def extractNodes(node):
    if node==None:
        return "";
    if node['_type']=='Name':
        return str(node["id"]);
    
    elif node['_type']=='Num':
        return str(node['n']);

    elif node['_type']=='Str':
        return "\""+str(node['s'])+"\"";

    elif node['_type']=='NameConstant':
        return str(node['value']);

    elif node['_type']=='Constant':
        if str(type(node['value']))=='<class \'str\'>':
            return "\""+str(node['value'])+"\"";
        else:
            return str(node['value']);
        
    elif node['_type']=='Starred':
        return "*"+str(extractNodes(node['value']));
    elif node['_type']=="Subscript":
        return str(extractNodes(node['value']))+'['+str(extractNodes(node['slice']))+"]";

    elif node['_type']=='Index':
        return extractNodes(node['value']);

    elif node['_type']=='Slice':
        sliceStr=str(extractNodes(node['lower']))+":"+str(extractNodes(node['upper']));
        if str(node['step'])!='None':
            sliceStr+=":"+str(extractNodes(node['step']));

        return sliceStr;

    elif node['_type']=='Delete':
        return 'del '+str(extractNodes(node['targets'][0]));

    elif node['_type']=='Dict':
        index=0;
        dct='{';
        for k in node['keys']:
            if len(node['keys'])==index+1:
                dct+=" "+str(extractNodes(k))+" : "+str(extractNodes(node['values'][index]))+" ";
            else:
                dct+=" "+str(extractNodes(k))+" : "+str(extractNodes(node['values'][index]))+",";
            index+=1;
        return dct+"}";

    elif node['_type']=='List':
        lst='[';
        index=1;
        for item in node['elts']:
            if len(node['elts'])==index:
                lst+=" "+str(extractNodes(item))+" ";
            else:
                lst+=" "+str(extractNodes(item))+',';
            index+=1;

        return lst+"]";

    elif node['_type']=='Tuple':
        lst='(';
        index=1;
        for item in node['elts']:
            if len(node['elts'])==index:
                lst+=" "+str(extractNodes(item))+" ";
            else:
                lst+=" "+str(extractNodes(item))+',';
            index+=1;

        return lst+")";

    elif node['_type']=='Set':
        lst='{';
        index=1;
        for item in node['elts']:
            if len(node['elts'])==index:
                lst+=" "+str(extractNodes(item))+" ";
            else:
                lst+=" "+str(extractNodes(item))+',';
            index+=1;

        return lst+"}";

    elif node['_type']=='BinOp':
        exp='';
        if node['left']['_type']=='BinOp':
            exp=str(extractNodes(node['left']));
        else:
            exp=str(extractNodes(node['left']));
        
        exp+=" "+str(operators[node['op']['_type']])+" ";

        exp+=str(extractNodes(node['right']));
        return "("+exp+")";

    elif node['_type']=='UnaryOp':
        operand=str(operators[node['op']['_type']])+str(extractNodes(node['operand']));
        return operand;

    elif node['_type']=='Compare':
        compareStat=str(extractNodes(node['left']))+" ";
        compareStat+=operators[node['ops'][0]['_type']]+" ";        #may need to be changed
        compareStat+=str(extractNodes(node['comparators'][0]));
        return compareStat;

    elif node['_type']=='BoolOp':
        return "("+str(extractNodes(node['values'][0]))+" "+operators[node['op']['_type']]+" "+ str(extractNodes(node['values'][1]))+")";

    elif node['_type']=='Attribute':
        calledFunc="";
        calledFunc+=str(extractNodes(node['value']))+"."+str(node['attr']);
        return calledFunc;

    elif node['_type']=='Call':
        callName=str(extractNodes(node['func']))+"(";
        index=1;
        for arg in node['args']:
            if len(node['args'])==index:
                callName+=str(extractNodes(arg));
            else:
                callName+=str(extractNodes(arg))+",";
            index+=1;
            
        return callName+')';

    elif node['_type']=='Lambda':
        callName=str("lambda ");
        index=1;
        for arg in node['args']['args']:
            if len(node['args']['args'])==index:
                callName+=str(arg['arg'])+": ";
            else:
                callName+=str(arg['arg'])+", ";
            index+=1;
            
        return callName + str(extractNodes(node['body']));
        
def getAssignCondition(node):
    assignStat="";
    for target in node['targets']:
        assignStat+=str(extractNodes(target))+" = ";

    return assignStat+str(extractNodes(node['value']));

def getAugAssignCondition(node):
    return str(extractNodes(node['target']))+" "+operators[node['op']['_type']]+"= "+str(extractNodes(node['value']));

def getWhileConditionalStat(node):
    return str(extractNodes(node['test']));

def getForConditionalStat(node):
    return str(extractNodes(node['target']))+" in "+str(extractNodes(node['iter']));

def getBranchStat(node):
    return str(extractNodes(node['test']));

def getExpression(node):
    return str(extractNodes(node['value']));